# Ray Tune - Understanding Hyperparameter Tuning Concepts

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

Many of the [Ray RLlib](../ray-rllib/00-Ray-RLlib-Overview.ipynb) lessons used [Ray Tune](http://tune.io) to train policies. This meant we trained _parameters_ that defined the policies. Now we'll learn that Ray Tune was actually designed to determine the best _hyperparameters_ for the problem, before training to determine the _parameters_.

This lesson introduces the concepts of _Hyperparameter Tuning or Optimization_ (HPO) and works through a nontrivial example using Tune. 

See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 

A [Ray Summit Connect](https://anyscale.com/blog/videos-and-slides-for-the-second-ray-summit-connect-june-17-2020/) talk by the creator of Tune, Richard Liaw, provides an excellent overview of the challenges of hyperparameter tuning and how Tune addresses these challenges. Another recent webinar [Fast and efficient hyperparameter tuning with Ray Tune](https://www.anyscale.com/events/2021/10/20/fast-scalable-hyperparameter-tuning-ray-tune) complements the discussion here.

## What Are Hyperparameters?

In _supervised learning_, we train a model with labeled data so the model can properly label new data values. Everything about the model is defined by a set of _parameters_, such as the weights in a linear regression. 

In contrast, the _hyperparameters_<sup>1</sup> define structural details about the kind of model itself, like whether or not we are using a linear regression or what architecture is best for a neural network, etc. Other quantities considered hyperparameters include learning rates, discount rates, etc. If we want our training process and resulting model to work well, we first need to determine the optimal or near-optimal set of hyperparameters.

How do we determine the optimal hyperparameters? The most straightfoward approach is to perform a loop where we pick a candidate set of values from some reasonably inclusive list of possible values, train a model, compare the results achieved with previous loop iterations, and pick the set that performed best. This process is called _Hyperparameter Tuning_ or _Optimization_ (HPO).

This simple algorithm can quickly become very expensive, however. Training a single neural networks can be compute intensive and the space of all possible architectures is huge. Hence, much of the research in hyperparameter tuning, especially for neural networks, focuses on ways to optimize HPO, such as early stopping and pruning the search space when some combinations appear to perform poorly.

1. _Hyperparameter_ is often spelled _hyper parameter_ or _hyper-parameter_, but we'll use the spelling with no space or dash.

![](../images/tune/what-are-hyperparameters.png)

## A Simple Example: $k$-Means 

Let's start with a very simple example of HPO, finding $k$ in $k$-means. 

The $k$-means algorithm finds clusters in a data set. It's a canonical example of _unsupervised learning_, where information is extracted from a data set, rather than using labeled data to train a model for labelling new data, as in _supervised learning_. We won't discuss the algorithm details, but the essense of it involves a "guess" for the expected number of clusters, the $k$ value, then calculating $k$ centroids (the coordinates at the center), one per cluster, along with determining to which cluster each data point belongs. The details are in [$k$-means Wikipedia article](https://en.wikipedia.org/wiki/K-means_clustering). The following animation shows the algorithm in action for a two-dimensional data set where three clusters are evident.

![K-Means Convergence](../images/tune/K-means_convergence.gif)

(source: [Wikipedia](https://en.wikipedia.org/wiki/K-means_clustering). [Larger Image](https://en.wikipedia.org/wiki/K-means_clustering#/media/File:K-means_convergence.gif))

While it is easy to see the clusters in this two-dimensional data set, that won't be for arbitrary datasets, especially those with more than three dimensions. Hence, we should determine the best $k$ value by trying many values and picking the value that appears to be best. In this case, "best" would mean that we minimize the distances between the datapoints and centroids. 

With just one hyperparameter, this problem is comparatively simple and brute force calculations to find the optimal $k$ is usually good enough.

## HPO for Neural Networks

Where HPO really becomes a challenge is finding the right neural network architecture for your problem. Why are neural networks a challenge? Consider this image of a typical architecture:

![Typical Neural Network](../images/tune/hpo-neural-network-example.png)

Every number you see is a hyperparameter! So are the decisions about how many layers to have, what kind of layer to use for each layer, etc. The space of possible hyperparameters is enormous, too big to explore naively.

So called _neural architecture search_ (NAS) has become a research field in its own right, along with general research in optimizing HPO. 

## Introduction to Ray Tune

[Ray Tune](http://tune.io) is the Ray-based native library for hyperparameter tuning. Tune makes it nearly as easy to run distributed, parallelized HPO as it is to run trials on a single machine manually, one after the other. 

Tune is built as an extensible, pluggable framework, with built-in integrations for many frameworks, including [OpenAI Gym environments](https://gym.openai.com/envs/), [PyTorch](https://pytorch.org), [TensorFlow](http://tensorflow.org), and recently, [sci-kit learn](https://scikit-learn.org/stable/) (see [this recent blog post](https://medium.com/distributed-computing-with-ray/gridsearchcv-2-0-new-and-improved-ee56644cbabf)).

## How Tune Works

Before we get into using Tune, let's understand the some definitions, terms, and components. With this understanding, you will get an insight into what happens when you use Tune to search your hyperparameter space and optimize your process to select the best, optimized model.

## Definitions

After a short preview of the [Tune basic steps and concepts](01-Ray-Tune-Warmup.ipynb) in our warm up tutorial, let's get an intuition of what those terms and steps mean.  

#### Trainable
This is your training function, with an objective function. As [trainable](https://docs.ray.io/en/latest/tune/api_docs/trainable.html?highlight=trainable#ray.tune.Trainable), it's one of the argument to `tune.run(...)` method. Tune offers two iterface APIs for trainable: functional and class.


#### Trial

A trial is an execution or run of a logical representation of a single hyperparameter configuration. Each trial is associated with an instance of a Trainable. And a collection of trials comprise an experiment.

#### Lifecycle of a trial¶
A trial’s life cycle consists of 6 stages:

Initialization (generation): A trial is first generated as a hyperparameter sample, and its parameters are configured according to what was provided in `tune.run` as part of the `config` arggument. Trials are then placed into a queue to be executed (with status PENDING).

**PENDING**: A pending trial is a trial to be executed on the machine. Every trial is configured with resource values. Whenever the trial’s resource values are available, tune will run the trial (by starting a ray actor holding the config and the training function).

**RUNNING**: A running trial is assigned a Ray Actor. There can be multiple running trials in parallel.

**ERRORED**: If a running trial throws an exception, Tune will catch that exception and mark the trial as errored. Note that exceptions can be propagated from an actor to the main Tune driver process. If `max_retries` is set, Tune will set the trial back into “PENDING” and later start it from the last checkpoint.

**TERMINATED**: A trial is terminated if it is stopped or finished by a Stopper/Scheduler. If using the Function API, the trial is also terminated when the function stops.

**PAUSED**: A trial can be paused by a Trial scheduler. This means that the trial’s actor will be stopped too. A paused trial can later be resumed from the most recent checkpoint.


#### Driver/worker process

The driver process is the python process that calls `tune.run` (which calls ray.init() underneath the hood); therefore, you
do not need to invoke `ray.init(...)` explicity. Tune does it for you during its inital run. The Tune's driver process runs on the node where you run your script (which calls `tune.run`), while Ray Tune trainable “actors” run on any node (either on the same node on multiple cores) or on worker nodes (with multiple cores on a distributed Ray cluster).

#### Ray Actors

Tune uses Ray Actors as worker node's processes to evaluate multiple Trainables in parallel.

[Ray Actors](https://docs.ray.io/en/latest/actors.html#actor-guide) allow you to parallelize an instance of a class in Python. When you instantiate a class that is a Ray actor, Ray will start a instance of that class on a separate process either on the same machine (or another distributed machine, if running a Ray cluster). This actor can then asynchronously execute method calls and maintain its own internal state.

### The execution of a trainable¶
Tune uses Ray actors to parallelize the evaluation of multiple hyperparameter configurations. Each actor is a Python process that executes an instance of the user-provided Trainable. The definition of the user-provided Trainable will be [serialized via cloudpickle](https://docs.ray.io/en/latest/serialization.html#serialization-guide) and sent to each actor process. Each Ray actor will start an instance of the Trainable to be executed.

If the Trainable is a class, it will be executed iteratively by calling train/step. After each invocation, the driver is notified that a “result dict” is ready. The driver will then pull the result via `ray.get`.

If the trainable is a callable or a function, it will be executed on the Ray actor process on a separate execution thread. Whenever `tune.report` is called, the execution thread is paused and waits for the driver to pull a result. After pulling, the actor’s execution thread will automatically resume.

The diagram below depicts how Tune launches trainables on the worker nodes as processes in which the the trainables are run. 
Each trial will have its own instance of a trainable, hence we parallelize trials and its respective configuration across cores on a worker. 

![](../images/ray_tune_report_launch_trainables.png)

Whenever the trainble calls `tune.report`, the driver will pull the metrics via `ray.get`, as shown in the diagram below.

![](../images/ray_tune_report_metrics.png)


Tune also integrates implementations of many state-of-the-art [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html) and [schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html), so it is easy to optimize your HPO process.

<hr style="border:3px solid gray"> </hr>

In [1]:
import ray
from ray import tune

Now we start Ray in this "driver" program (notebook).

In [2]:
ray.init(ignore_reinit_error=True)

2022-02-21 11:39:57,733	INFO services.py:1376 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-02-21_11-39-55_279085_13161/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-21_11-39-55_279085_13161/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-02-21_11-39-55_279085_13161',
 'metrics_export_port': 57721,
 'gcs_address': '127.0.0.1:54836',
 'node_id': 'b70e7343ebd5917e42c3f818d1447803ab4681c6809a0510d9e8d372'}

In our [RLlib Simple Multi-Armed Bandit](../ray-rllib/multi-armed-bandits/03-Simple-Multi-Armed-Bandit.ipynb) lesson, we used Tune to train RLlib, but not train hyperparameters. 

> **NOTE**: If you have not gone through the RLlib tutorial yet, you may skip the rest of this tutorital, for it takes about 5-6 minutes. The message to take away here is that Tune provides many search algorithms and schedulers that can be specified as arguments. The next three tutoirals shows you how.

Now we'll use Tune to train hyperparameters. We'll use the same experimental environment we used in our [RLlib `CartPole` lesson](../ray-rllib/explore-rllib/01-Application-Cart-Pole.ipynb). 

> **Aside:** In case you haven't gone through the [Ray RLlib tutorial](../ray-rllib/00-Ray-RLlib-Overview.ipynb), [CartPole](https://gym.openai.com/envs/CartPole-v1/) is an [OpenAI Gym](https://gym.openai.com/envs/) environment that simulates a cart that moves left or right while attempting to balance a vertical pole. A _policy_ is trained to optimize changing the velocity up and down to control right and left movement with the goal of keeping the pole balanced for as long as possible.

The most important hyperparameters for `CartPole` are for the neural network used to learn how to balance the pole. A simple network suffices. We'll use a fully-connected network with two hidden layers, but use Tune to find an optimal choice for the sizes of the layers. To keep the computation tractable for our purposes, we'll just pick sizes from the list of 20 and 40. 

It's probably that some specific number between these 10 and 100 is optimal for one layer and a different number is optimal for the other layer. However, the computation required for trying all possible combinations is $O(n^2)$. You should consider trying more numbers if you don't mind waiting!

However, the two we've chosen are good enough as we'll see. We'll consider what "good enough" means when we look at the results.

The next cell runs Tune for this purpose. The comments explain what each argument does. We'll do four tries, one for each combination of the two possible values for the two hidden layers.

> **Note:** `tune.run` will handle Ray initialization for us, if it isn't already initialized. To force Tune to throw an error instead, pass the argument `ray_auto_init=False`.

The next cell will take 5-6 minutes to run.

In [3]:
analysis = tune.run(
    "PPO",                                    # Use proximal policy optimization to train 
    stop={"episode_reward_mean": 400},        # Stopping criteria, when average reward over the episodes
                                              # of training equals 400 out of a maximum possible 500 score.
    config={
        "env": "CartPole-v1",                 # Tune can associate this string with the environment.
        "num_gpus": 0,                        # If you have GPUs, go for it!
        "num_workers": 6,                     # Number of Ray workers to use; Use one LESS than 
                                              # the number of cores you want to use (or omit this argument)!
        "model": {                            # The NN model we'll optimize.
            'fcnet_hiddens': [                # "Fully-connected network with N hidden layers".
                tune.grid_search([20, 40]),   # Try these four values for layer one.
                tune.grid_search([20, 40])    # Try these four values for layer two.
            ]
        },
    },
    verbose=1
)

2022-02-21 11:43:46,044	INFO tune.py:636 -- Total run time: 219.32 seconds (218.17 seconds for the tuning loop).
